In [25]:
from casadi import *

In [26]:
# Set up the problem
N = 100 # number of control intervals
opti = Opti() # optimization problem

# Declare the decision variables
X = opti.variable(2, N+1) # State trajectory
pos = X[0,:]
speed = X[1,:]
U = opti.variable(1,N) # control trajectory (throttle) 
T = opti.variable() # final time

In [27]:
# Set up the objective
opti.minimize(T) # race in minimal time

# Specify system dynamics
def f(x, u):
    dx = MX.sym('dx')
    dx = [x[1], u- x[1]]
    return dx

#f = lambda x, u: [x[1], u - x[1]]

In [28]:
dt = T/N # Control interval
for k in range(N): # Loop over control intervals
    k1 = MX.sym('k1')
    k1 = f(X[:,k], U[:,k])
    print("k1 type ", type(k1), "dt type ", type(dt))
    k2 = f(X[:,k] + dt/2*k1, U[:,k])
    k3 = f(X[:,k] + dt/2*k2, U[:,k])
    k4 = f(X[:,k] + dt*k3, U[:,k])
    x_next = X[:,k] + dt/6*(k1+2*k2+2*k3+k4)
    opti.subject_to(X[:,k+1] == x_next)

k1 type  <class 'list'> dt type  <class 'casadi.casadi.MX'>


TypeError: can't multiply sequence by non-int of type 'MX'

In [ ]:
import casadi as ca

dt = T / N  # length of a control interval

for k in range(N):  # loop over control intervals
    # Runge-Kutta 4 integration
    k1 = f(X[:, k], U[:, k])
    k2 = f(X[:, k] + dt / 2 * k1, U[:, k])
    k3 = f(X[:, k] + dt / 2 * k2, U[:, k])
    k4 = f(X[:, k] + dt * k3, U[:, k])
    x_next = X[:, k] + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    opti.subject_to(ca.vertcat(X[:, k + 1]) == x_next)  # close the gaps
